In [2]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

Using Theano backend.


In [3]:
# start with each input secion maps to one note
# train model for that (just list comprhension on noets for contains 1 maps to true)

# then move to bars eg section of 4 bars maps to output for each note
# error = probability of note being true vs was it really

# try bar + prev notes (home use weird dimensioned data?) to predict next notes

# try feeding in non structured data (bpm, position of time in song, song length, 
# things about feel of song (generated features))
# try using keras merge layer to add extra features

In [79]:
def has_step(row):
    return '1' in row or '2' in row or '4' in row

def build_rows_generator():
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    save_files = listdir('data')
    for song_data in songs_to_use[:100]:
        if '{0}_beat_features.csv'.format(song_data[0]) in save_files and '{0}_notes.csv'.format(song_data[0]) in save_files:
            beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(song_data[0])).values
            notes = pd.read_csv('data/{0}_notes.csv'.format(song_data[0]), converters={'0': lambda x: str(x)}).values
            beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
            for i in range(min(len(notes), len(beat_features))):
                X = beat_features[i]
                y = has_step(notes[i][0])
                yield (X, y)

In [80]:
beat_feature_model = Sequential()
beat_feature_model.add(Dense(32, input_dim=40, init='uniform', activation='relu'))
beat_feature_model.add(Dropout(0.5))
beat_feature_model.add(Dense(32, activation='relu'))
beat_feature_model.add(Dropout(0.5))
beat_feature_model.add(Dense(1, activation='sigmoid'))

beat_feature_model.compile(loss='binary_crossentropy',
                           optimizer='rmsprop',
                           metrics=['accuracy'])

In [81]:
gen = build_rows_generator()
beat_feature_model.fit_generator(gen, samples_per_epoch=100, nb_epoch=10)

Epoch 1/10


Exception: Error when checking model input: expected dense_input_13 to have shape (None, 40) but got array with shape (40, 1)

In [75]:
gen = build_rows_generator()
a = [next(gen) for i in range(5)]
#beat_feature_model.predict(a, batch_size=1, verbose=0)
np.array(a[1][0]).shape[0]

40

In [ ]:
# remember https://github.com/fchollet/keras/issues/2708
# https://keras.io/getting-started/sequential-model-guide/#training

In [ ]:
beat_feature_model = Sequential([
    Dense(32, input_dim=784),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

misc_feature_model = Sequential([
    Dense(32, input_dim=4),
    Activation('relu'),
])

model = Sequential([
    Merge([beat_feature_model, misc_feature_model], mode='concat'),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def false_rates(y_true, y_pred):
    false_neg = ...
    false_pos = ...
    return {
        'false_neg': false_neg,
        'false_pos': false_pos,
    }

model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy', mean_pred, false_rates])

model.fit(data, labels, nb_epoch=10, batch_size=32)
model.predict(self, x, batch_size=32, verbose=0)